In [1]:
import os
import pandas as pd
import shutil
import imageio
import imgaug as ia
import matplotlib.pyplot as plt
import matplotlib.image as img

from imgaug import augmenters as iaa

## Input

In [2]:
# for data cleaning
data = "annotations/covid-severity-scores.csv"
raw_image_dir = "images"
error_file = ['ciaa199.pdf-001-a.png' , 
              'ciaa199.pdf-001-b.png', 
              'ciaa199.pdf-001-c.png', 
              '6C94A287-C059-46A0-8600-AFB95F4727B7.jpeg', 
              '53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg', 
              'covid-19-rapidly-progressive-acute-respiratory-distress-syndrome-ards-admission.jpg', 
              'covid-19-pneumonia-28.png']

In [3]:
# for augmentation
function_names = ["Gamma", "CLAHE"]
output_dir = "augmented"
output_csv_name = "augmented_covid-severity-scores.csv"

## Raw data cleaning
You need to prepare the following contents first:
* data -> raw covid-severity-scores.csv
* error_file -> you need to specify which files are needed to be deleted
* raw_image_dir -> 原圖群資料夾

In [4]:
df = pd.read_csv(data)
print(df.head())
# delete description data
df = df.iloc[5:,:]
#df.head()
# data cleaning

df_cleaned = df[~df.iloc[:,0].isin(error_file)]

  ## COVID-19 Pneumonia Severity Scores (Mean of 3 expert raters)  \
0  ## License: CC BY-SA Creative Commons Attribut...                
1  ## Citation: Joseph Paul Cohen et al. Predicti...                
2  ## geographic_extent_mean: The extent of lung ...                
3  ## opacity_mean: The degree of opacity. The to...                
4                                           filename                

        Unnamed: 1    Unnamed: 2  
0              NaN           NaN  
1              NaN           NaN  
2              NaN           NaN  
3              NaN           NaN  
4  geographic_mean  opacity_mean  


## data augmentation
You need to prepare the following contents first:
* function_names -> 要選用的擴增功能"們"
* output_dir -> 產圖, csv 的資料夾

In [5]:
output_csv = df.copy()
output_csv.columns = ["name", "geographic_mean", "opacity_mean"]

if not os.path.exists(output_dir):
        os.makedirs(output_dir)

total_dir = os.path.join(output_dir, "total")
if not os.path.exists(total_dir):
        os.makedirs(total_dir)
        
save_dir = os.path.join(output_dir, "original")
if not os.path.exists(save_dir):
        os.makedirs(save_dir)
for data in df_cleaned.iloc[:,0].tolist():
    shutil.copy(os.path.join(raw_image_dir, data), save_dir)
    shutil.copy(os.path.join(raw_image_dir, data), total_dir)

for function_name in function_names:
    save_dir = os.path.join(output_dir, function_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for index, row in df_cleaned.iterrows():
        data = row[0]
        geographic_mean = row[1]
        opacity_mean = row[2]
        data_name, ext = os.path.splitext(data)
        data_path = os.path.join(raw_image_dir, data)
        image = imageio.imread(data_path)

        # aug function select
        if function_name == "CLAHE":
            function = iaa.CLAHE()
        elif function_name == "Gamma":
            function = iaa.GammaContrast((0.5, 2.0))

        image_aug = function(image=image)
        newPath = os.path.join(save_dir, data_name+f"_{function_name}"+ext)
        new = pd.DataFrame([[data_name+f"_{function_name}"+ext, geographic_mean, opacity_mean]], columns=["name", "geographic_mean", "opacity_mean"])
        output_csv.append(new)
        imageio.imsave(newPath, image_aug)
        shutil.copy(newPath, total_dir)

In [6]:
output_csv.to_csv(os.path.join(output_dir,output_csv_name))